In [21]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from typing import Dict, Callable

In [22]:
def setup_model_and_tokenizer(model_name: str = 'gpt2'):
    """
    Loads a pretrained GPT-2 model and its tokenizer.
    """
    print(f"Loading model and tokenizer for '{model_name}'...")
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    model.eval()
    return model, tokenizer

In [23]:
def get_task_datasets() -> Dict[str, Dict]:
    """
    Defines the clean and corrupted prompts and answers for each task.
    """
    datasets = {
        "analogy": {
            "description": "Letter-String Analogy Task ('+1' vs No Rule)",
            "clean_prompt": "ABCD:ABCE::JKLM:JKL",
            "clean_correct_answer": "N",
            "clean_incorrect_answer": "M",
            "corrupted_prompt": "ABCD:ABCD::JKLM:JKL",
            "corrupted_correct_answer": "M",
            "corrupted_incorrect_answer": "N",
        },
        "sequencing": {
            "description": "Next-Item Sequencing Task",
            "clean_prompt": "January:February::Wednesday:",
            "clean_correct_answer": "Thursday",
            "clean_incorrect_answer": "Wednesday",
            "corrupted_prompt": "January:January::Wednesday:",
            "corrupted_correct_answer": "Wednesday",
            "corrupted_incorrect_answer": "Thursday",
        }
    }
    return datasets

In [24]:
activation_cache: Dict[str, torch.Tensor] = {}

def caching_hook_factory(hook_name: str) -> Callable:
    def hook(module, input, output):
        tensor_to_cache = output[0] if isinstance(output, tuple) else output
        activation_cache[hook_name] = tensor_to_cache.detach()
    return hook

def patching_hook_factory(hook_name: str, head_index: int = None, d_head: int = None) -> Callable:
    """Factory for patching hooks. FIX: Handles mismatched sequence lengths."""
    def hook(module, input, output):
        if hook_name not in activation_cache:
            raise ValueError(f"Activation for {hook_name} not found in cache!")
        cached_activation = activation_cache[hook_name]

        patched_output = output.clone()
        
        # Determine the minimum sequence length to avoid shape mismatch errors.
        # The sequence length is the second-to-last dimension.
        min_seq_len = min(patched_output.shape[-2], cached_activation.shape[-2])

        if head_index is not None: # Attention head patch
            start, end = head_index * d_head, (head_index + 1) * d_head
            if patched_output.ndim == 3:
                # Patch for [batch, seq_len, embed_dim]
                patched_output[:, :min_seq_len, start:end] = cached_activation[:, :min_seq_len, start:end]
            elif patched_output.ndim == 2:
                # Patch for [seq_len, embed_dim]
                patched_output[:min_seq_len, start:end] = cached_activation[:min_seq_len, start:end]
        else: # MLP patch
            if patched_output.ndim == 3:
                # Patch for [batch, seq_len, embed_dim]
                patched_output[:, :min_seq_len, :] = cached_activation[:, :min_seq_len, :]
            elif patched_output.ndim == 2:
                # Patch for [seq_len, embed_dim]
                patched_output[:min_seq_len, :] = cached_activation[:min_seq_len, :]
        
        return patched_output
    return hook

In [25]:
def get_module_by_name(model: nn.Module, name: str) -> nn.Module:
    for part in name.split('.'):
        model = getattr(model, part)
    return model

In [26]:
def run_with_hooks(
        model: nn.Module,
        tokenizer: GPT2Tokenizer,
        prompt: str,
        hook_fns: Dict[str, Callable],
) -> torch.Tensor:
    handles = []
    try:
        for name, hook_fn in hook_fns.items():
            module_name = name.split('_')[0] if 'attn.c_proj' in name else name
            module = get_module_by_name(model, name)
            handles.append(module.register_forward_hook(hook_fn))
        inputs = tokenizer(prompt, return_tensors='pt', padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.logits[0, -1, :]
    finally:
        for handle in handles:
            handle.remove()

In [27]:
def calculate_logit_diff(
        logits: torch.Tensor,
        tokenizer: GPT2Tokenizer,
        correct_answer: str,
        incorrect_answer: str
) -> float:
    correct_id = tokenizer.encode(correct_answer, add_prefix_space=False)[0]
    incorrect_id = tokenizer.encode(incorrect_answer, add_prefix_space=False)[0]
    return (logits[correct_id] - logits[incorrect_id]).item()

In [28]:
def perform_patching_experiment(
        model: nn.Module,
        tokenizer: GPT2Tokenizer,
        source_prompt: str,
        dest_prompt: str,
        dest_correct_answer: str,
        dest_incorrect_answer: str,
        layer: int,
        component_type: str,
        head_index: int = None
) -> float:
    global activation_cache
    activation_cache = {}

    if component_type == 'attn_head':
        hook_name = f"transformer.h.{layer}.attn.c_proj"
    elif component_type == 'mlp':
        hook_name = f"transformer.h.{layer}.mlp.c_proj"
    else:
        raise ValueError("Invalid component type")

    caching_hooks = {hook_name: caching_hook_factory(hook_name)}
    run_with_hooks(model, tokenizer, source_prompt, caching_hooks)

    d_head = model.config.n_embd // model.config.n_head if component_type == 'attn_head' else None
    patching_hooks = {hook_name: patching_hook_factory(hook_name, head_index, d_head)}
    patched_logits = run_with_hooks(model, tokenizer, dest_prompt, patching_hooks)

    return calculate_logit_diff(patched_logits, tokenizer, dest_correct_answer, dest_incorrect_answer)

def run_exploratory_sweep(
        model: nn.Module,
        tokenizer: GPT2Tokenizer,
        task_data: Dict,
        patch_type: str
) -> pd.DataFrame:
    n_layers = model.config.n_layer
    n_heads = model.config.n_head

    if patch_type == 'noising':
        source_prompt = task_data['corrupted_prompt']
        dest_prompt = task_data['clean_prompt']
        dest_correct_answer = task_data['clean_correct_answer']
        dest_incorrect_answer = task_data['clean_incorrect_answer']
    elif patch_type == 'denoising':
        source_prompt = task_data['clean_prompt']
        dest_prompt = task_data['corrupted_prompt']
        dest_correct_answer = task_data['corrupted_correct_answer']
        dest_incorrect_answer = task_data['corrupted_incorrect_answer']
    else:
        raise ValueError("patch_type must be 'noising' or 'denoising'")

    clean_logits = run_with_hooks(model, tokenizer, task_data['clean_prompt'], {})
    clean_baseline = calculate_logit_diff(clean_logits, tokenizer, task_data['clean_correct_answer'], task_data['clean_incorrect_answer'])

    corrupted_logits = run_with_hooks(model, tokenizer, task_data['corrupted_prompt'], {})
    corrupted_baseline = calculate_logit_diff(corrupted_logits, tokenizer, task_data['corrupted_correct_answer'], task_data['corrupted_incorrect_answer'])

    print(f"\nRunning {patch_type} sweep for '{task_data['description']}'")
    print(f"  - Clean Run Baseline Logit Diff: {clean_baseline:.4f}")
    print(f"  - Corrupted Run Baseline Logit Diff: {corrupted_baseline:.4f}")

    results = []
    print("  - Patching Attention Heads...")
    for layer in range(n_layers):
        for head in range(n_heads):
            patched_logit_diff = perform_patching_experiment(
                model, tokenizer, source_prompt, dest_prompt,
                dest_correct_answer, dest_incorrect_answer,
                layer, 'attn_head', head
            )
            results.append({'layer': layer, 'head': head, 'type': 'attn_head', 'patched_logit_diff': patched_logit_diff})

    print("  - Patching MLP Layers...")
    for layer in range(n_layers):
        patched_logit_diff = perform_patching_experiment(
            model, tokenizer, source_prompt, dest_prompt,
            dest_correct_answer, dest_incorrect_answer,
            layer, 'mlp'
        )
        results.append({'layer': layer, 'head': -1, 'type': 'mlp', 'patched_logit_diff': patched_logit_diff})

    df = pd.DataFrame(results)

    if patch_type == 'noising':
        df['effect'] = df['patched_logit_diff'] - clean_baseline
    else:
        df['effect'] = df['patched_logit_diff'] - corrupted_baseline

    return df

In [29]:
def plot_results(df: pd.DataFrame, title: str, component_type: str, output_path: str = None):
    """Plots a heatmap or bar chart using Matplotlib and optionally saves it."""
    if component_type == 'attn_head':
        pivot_df = df[df['type'] == 'attn_head'].pivot(index='head', columns='layer', values='effect')
        fig, ax = plt.subplots(figsize=(12, 10))
        # Center the color map around 0
        max_abs_val = pivot_df.abs().max().max()
        im = ax.imshow(pivot_df, cmap='coolwarm', vmin=-max_abs_val, vmax=max_abs_val, aspect='auto')
        # Create colorbar
        cbar = ax.figure.colorbar(im, ax=ax)
        cbar.ax.set_ylabel("Effect on Logit Difference", rotation=-90, va="bottom")
        # Set ticks and labels
        ax.set_xticks(np.arange(pivot_df.shape[1]))
        ax.set_yticks(np.arange(pivot_df.shape[0]))
        ax.set_xticklabels(pivot_df.columns)
        ax.set_yticklabels(pivot_df.index)
        ax.set_xlabel("Layer")
        ax.set_ylabel("Head Index")
    elif component_type == 'mlp':
        mlp_df = df[df['type'] == 'mlp'].sort_values('layer')
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.bar(mlp_df['layer'], mlp_df['effect'], color='skyblue')
        ax.set_xlabel("Layer")
        ax.set_ylabel("Average Effect on Logit Difference")
        ax.grid(axis='y', linestyle='--')
        # Set x-ticks to be integers
        ax.set_xticks(mlp_df['layer'])
    else:
        raise ValueError("Invalid component type")

    ax.set_title(title)
    fig.tight_layout()
    if output_path:
        plt.savefig(output_path, format='pdf')
        print(f"Saved plot to: {output_path}")
    plt.show()
    plt.close(fig)

In [30]:
if __name__ == '__main__':
    MODEL_NAME = 'gpt2'
    model, tokenizer = setup_model_and_tokenizer(MODEL_NAME)
    datasets = get_task_datasets()

    all_results = {}

    for task_name, task_data in datasets.items():
        for patch_type in ['noising', 'denoising']:
            result_key = f"{task_name}_{patch_type}"
            df = run_exploratory_sweep(model, tokenizer, task_data, patch_type)
            all_results[result_key] = df

    # --- Save Dataframes to CSV ---
    print("\n--- Saving Results to CSV ---")
    output_dir_data = "patching_results_data_3"
    if not os.path.exists(output_dir_data):
        os.makedirs(output_dir_data)

    for result_key, df in all_results.items():
        file_path = os.path.join(output_dir_data, f"{result_key}_results.csv")
        df.to_csv(file_path, index=False)
        print(f"Saved data to: {file_path}")

    # --- Visualize and Save Plots to PDF ---
    print("\n--- Generating and Saving Plots ---")
    output_dir_plots = "patching_results_plots_3"
    if not os.path.exists(output_dir_plots):
        os.makedirs(output_dir_plots)

    for result_key, df in all_results.items():
        task_name, patch_type = result_key.split('_')

        plot_path_attn = os.path.join(output_dir_plots, f"{result_key}_attn_heads.pdf")
        plot_path_mlp = os.path.join(output_dir_plots, f"{result_key}_mlp_layers.pdf")

        title_attn = f"Attention Heads Effect ({patch_type.capitalize()})\n{datasets[task_name]['description']}"
        plot_results(df, title_attn, 'attn_head', output_path=plot_path_attn)

        title_mlp = f"MLP Layers Effect ({patch_type.capitalize()})\n{datasets[task_name]['description']}"
        plot_results(df, title_mlp, 'mlp', output_path=plot_path_mlp)

    print("\n--- End of Experiment ---")

Loading model and tokenizer for 'gpt2'...

Running noising sweep for 'Letter-String Analogy Task ('+1' vs No Rule)'
  - Clean Run Baseline Logit Diff: 2.0557
  - Corrupted Run Baseline Logit Diff: -2.3486
  - Patching Attention Heads...
  - Patching MLP Layers...

Running denoising sweep for 'Letter-String Analogy Task ('+1' vs No Rule)'
  - Clean Run Baseline Logit Diff: 2.0557
  - Corrupted Run Baseline Logit Diff: -2.3486
  - Patching Attention Heads...
  - Patching MLP Layers...

Running noising sweep for 'Next-Item Sequencing Task'
  - Clean Run Baseline Logit Diff: -0.1954
  - Corrupted Run Baseline Logit Diff: 0.0659
  - Patching Attention Heads...
  - Patching MLP Layers...

Running denoising sweep for 'Next-Item Sequencing Task'
  - Clean Run Baseline Logit Diff: -0.1954
  - Corrupted Run Baseline Logit Diff: 0.0659
  - Patching Attention Heads...
  - Patching MLP Layers...

--- Saving Results to CSV ---
Saved data to: patching_results_data_3/analogy_noising_results.csv
Saved 

/scratch-local/awadehra.12342160/ipykernel_2627508/3715323595.py:36: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/scratch-local/awadehra.12342160/ipykernel_2627508/3715323595.py:36: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Saved plot to: patching_results_plots_3/analogy_denoising_attn_heads.pdf
Saved plot to: patching_results_plots_3/analogy_denoising_mlp_layers.pdf


/scratch-local/awadehra.12342160/ipykernel_2627508/3715323595.py:36: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/scratch-local/awadehra.12342160/ipykernel_2627508/3715323595.py:36: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Saved plot to: patching_results_plots_3/sequencing_noising_attn_heads.pdf
Saved plot to: patching_results_plots_3/sequencing_noising_mlp_layers.pdf


/scratch-local/awadehra.12342160/ipykernel_2627508/3715323595.py:36: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/scratch-local/awadehra.12342160/ipykernel_2627508/3715323595.py:36: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Saved plot to: patching_results_plots_3/sequencing_denoising_attn_heads.pdf
Saved plot to: patching_results_plots_3/sequencing_denoising_mlp_layers.pdf

--- End of Experiment ---


/scratch-local/awadehra.12342160/ipykernel_2627508/3715323595.py:36: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/scratch-local/awadehra.12342160/ipykernel_2627508/3715323595.py:36: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
